In [1]:
import fsl

In [2]:
import ants

In [3]:
import os
os.environ["TMPDIR"] = "/workspace/temp"


In [4]:
import nibabel as nib
from deepbrain import Extractor
import os
import pandas as pd
import numpy as np
from pathlib import Path
import time

In [ ]:
patients = pd.read_csv("idaSearch_7_07_2021 (3).csv")

def strip(file, output, name):
    img = nib.load(file).get_fdata().squeeze()
    affine = nib.load(file).affine
    now = time.time()
    ext = Extractor()
    prob = ext.run(img) 
    mask = prob >= 0
    stripped = (1 * mask).astype(np.uint8)
    stripped = nib.Nifti1Image(stripped, affine)
  
    brain = img[:]
    brain[~mask] = 0
    brain = nib.Nifti1Image(brain, affine)
    nib.save(brain, "petbrains/" + str(name + ".nii.gz"))
    #nib.save(stripped, str(name + "_mask.nii.gz"))
    print("Extraction time: {0:.2f} secs.".format(time.time() - now))
    
def strippet(file, output, name):
    img = nib.load(file).get_fdata().squeeze()
    affine = nib.load(file).affine
    now = time.time()
    #ext = Extractor()
    #prob = ext.run(img) 
    mask = img >= .5
    stripped = (1 * mask).astype(np.uint8)
    stripped = nib.Nifti1Image(stripped, affine)
   
    brain = img[:]
    brain[~mask] = 0
    brain = nib.Nifti1Image(brain, affine)
    nib.save(brain, "petbrains/" + str(name + ".nii.gz"))
    #nib.save(stripped, str(name + "_mask.nii.gz"))
    print("Extraction time: {0:.2f} secs.".format(time.time() - now))
    
strippet("data/new_alzheimers/ADNI/002_S_5018/Coreg,_Avg,_Standardized_Image_and_Voxel_Size/2012-11-14_08_14_11.0/I347757/ADNI_002_S_5018_PT_Coreg,_Avg,_Standardized_Image_and_Voxel_Size_Br_20121126135446331_69_S174525_I347757.nii", "petbrains", "347757")

In [ ]:
PET = patients.loc[(patients["Modality"] == "PET")]["Image ID"].values

for root, directories, files in os.walk("./data"):
    for file in files:
        if file.endswith("brain.nii.gz"):
            pass
        #elif Path(root+"brain.nii.gz").exists():
        #    pass
        elif int(file.split("_I")[-1][:-4]) in PET:
           # print(int(file.split("_I")[-1][:-4]))
            strippet(os.path.join(root, file), root, file.split("_I")[-1][:-4])

In [5]:
from fsl.wrappers.flirt import flirt
def register(image_id, size="1"):
    brain = ants.image_read("brains/" + image_id + ".nii.gz")
    #mask = ants.image_read("masks/" + image_id+"_mask.nii.gz")
    ref = ants.image_read("MNI152_T1_1mm_brain.nii.gz")
    reg = "reg" if size == 1 else f"reg{size}"
    #out = Path("registered/"+image_id + f"brain_{reg}.nii.gz")
    #mat = Path("registered/"+image_id + f"brain_{reg}.mat")
    #full = Path("registered/"+image_id + f"full_{reg}.nii.gz")
    #mask_out = Path("registered/"+image_id + f"brain_mask_{reg}.nii.gz")
                   
    now = time.time()
    print("begin", flush = True)
    registered = ants.registration(ref, brain, type_of_transform = "SyN")
    print("registered", flush = True)

    output = ants.apply_transforms(ref, brain, registered['fwdtransforms'])
    print("Registration time: {0:.2f} secs.".format(time.time() - now), flush = True)
    nib.save(ants.utils.convert_nibabel.to_nibabel(output), "registered/" + image_id + "_registered.nii.gz")#     if not full.exists():
#         now = time.time() 
#         flirt(src,ref,out=full,applyxfm=True,init=mat,v=True)
#         print("Full head time: {0:.2f} secs.".format(time.time() - now))
        
#     if not mask_out.exists():
#         now = time.time()
#         flirt(mask.as_posix(),ref,out=mask_out,applyxfm=True,init=mat,v=True)
#         print("Mask time: {0:.2f} secs.".format(time.time() - now))

In [6]:
alz = pd.read_csv("alzheimers.csv").drop_duplicates()
conv = pd.read_csv("converted.csv").drop_duplicates()
non = pd.read_csv("nonconverted.csv").drop_duplicates()
cont = pd.read_csv("control.csv").drop_duplicates()

In [9]:
for ids in alz["MRI_ID"]:
    if Path("registered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in cont["MRI_ID"]:
    if Path("registered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in conv["MRI_ID"]:
    if Path("registered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in non["MRI_ID"]:
    if Path("registered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

alz["MRI_ID"].size + cont["MRI_ID"].size + conv["MRI_ID"].size + non["MRI_ID"].size

something went wrong with 74612
something went wrong with 36567
something went wrong with 56317
something went wrong with 298812
something went wrong with 356625
something went wrong with 383217
something went wrong with 60437
something went wrong with 65598
something went wrong with 32235
something went wrong with 35484
something went wrong with 32005
something went wrong with 298816
something went wrong with 334757
something went wrong with 366687
something went wrong with 334763
something went wrong with 63404
something went wrong with 63413
something went wrong with 356638
something went wrong with 38654
something went wrong with 38659
something went wrong with 65246
something went wrong with 62936
something went wrong with 306756
something went wrong with 39922
something went wrong with 63173
something went wrong with 67221
something went wrong with 95660
something went wrong with 66076
something went wrong with 82579
something went wrong with 298893
something went wrong with 3566

564

In [ ]:
for ids in [306756]:
    brain = ants.image_read("brains/" + str(ids) + ".nii.gz")
    ref = ants.image_read("MNI152_T1_1mm_brain.nii.gz")
    reg = "reg"
    now = time.time()
    print("begin", flush = True)
    registered = ants.registration(ref, brain, type_of_transform = "SyN")
    print("registered", flush = True)

    output = ants.apply_transforms(ref, brain, registered['fwdtransforms'])
    print("Registration time: {0:.2f} secs.".format(time.time() - now), flush = True)
    nib.save(ants.utils.convert_nibabel.to_nibabel(output), "registered/" + str(ids) + "_registered.nii.gz")
    print("Extraction time: {0:.2f} secs.".format(time.time() - now))


begin


In [ ]:
from fsl.wrappers.flirt import flirt
def register(image_id, size="1"):
    brain = ants.image_read("petbrains/" + image_id + ".nii.gz")
    #mask = ants.image_read("masks/" + image_id+"_mask.nii.gz")
    ref = ants.image_read("MNI152_T1_1mm_brain.nii.gz")
    reg = "reg" if size == 1 else f"reg{size}"
    out = Path("petregistered/"+image_id + f"brain_{reg}.nii.gz")
    mat = Path("petregistered/"+image_id + f"brain_{reg}.mat")
    full = Path("petregistered/"+image_id + f"full_{reg}.nii.gz")
    mask_out = Path("petregistered/"+image_id + f"brain_mask_{reg}.nii.gz")
                   
    if not mat.exists():
        now = time.time()
        registered = ants.registration(ref, brain, type_of_transform = "SyN")
        output = ants.apply_transforms(ref, brain, registered['fwdtransforms'])
        print("Registration time: {0:.2f} secs.".format(time.time() - now))
        nib.save(ants.utils.convert_nibabel.to_nibabel(output), "petregistered/" + image_id + "_registered.nii.gz")#     if not full.exists():
#         now = time.time() 
#         flirt(src,ref,out=full,applyxfm=True,init=mat,v=True)
#         print("Full head time: {0:.2f} secs.".format(time.time() - now))
        
#     if not mask_out.exists():
#         now = time.time()
#         flirt(mask.as_posix(),ref,out=mask_out,applyxfm=True,init=mat,v=True)
#         print("Mask time: {0:.2f} secs.".format(time.time() - now))

for ids in alz["PET_ID"]:
    if Path("petregistered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in cont["PET_ID"]:
    if Path("petregistered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in conv["PET_ID"]:
    if Path("petregistered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in non["PET_ID"]:
    if Path("petregistered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))


In [ ]:
img = nib.load("data/new_control/ADNI/007_S_4387/Coreg,_Avg,_Standardized_Image_and_Voxel_Size/2011-12-19_14_03_00.0/I274777/ADNI_007_S_4387_PT_Coreg,_Avg,_Standardized_Image_and_Voxel_Size_Br_20120102142716286_82_S134552_I274777.nii")

In [ ]:
img = img.get_fdata()

In [ ]:
img.max()

In [ ]:
root='./data/new_converts/ADNI'
convlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]


In [ ]:
root='./data/new_nonconverts/ADNI'
nonconvlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]


In [ ]:
len(set(convlist) & set(nonconvlist))

In [ ]:
root='./data/new_alzheimers/ADNI'
alzlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]


In [ ]:
root='./data/new_control/ADNI'
controllist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]


In [ ]:
set(controllist) & set(convlist)

In [ ]:
registered

In [12]:
    ref = ants.image_read("MNI152_T1_1mm_brain.nii.gz")
